## gcp_selection - Get image coordinates for GCPs
* Load an image w/ visibile GCP locations
* Use holoviews to pan, zoom, and click on GCP location
* Edit table to add gcp ids
* Save points in .csv file

Python implementation of CIRN QCIT function `gcpSelection.m`  

##### Required libraries  
  numpy  
  PIL  
  holoviews 

##### Notice: Don't run the whole notebook at once...user interaction is needed in cell 4.

csherwood@usgs.gov

#### Import required libaries

In [ ]:
import numpy as np
import holoviews as hv
from PIL import Image
from holoviews import opts, streams
from holoviews.plotting.links import DataLink
hv.extension('bokeh')

#### User input - edit this cell:

In [ ]:
# Specify camera id and calibration date
cam_id = 'caco-02_C2'
cal_date = '2023-03-22'

# Specify location for output
odir = './'                  
oname_base = '_gcp_UVd_intial.csv'

# Specify image location
img_dir = 'D:/crs/proj/2019_CACO_CoastCam/CACO-02/2023-03-22_extrinsics/images/'
img_fname = '1679497200.Wed.Mar.22_15_00_00.GMT.2023.caco-02.c2.timex.jpg'

#### ... end user input.
#### Check these values:

In [ ]:
oname = cam_id+'-'+cal_date+oname_base
opath = odir+oname
print('Output path: ',opath)

#### Pick gcp locations

Select pixel coordinates `Ud` and `Vd` in the "distorted" camera coordinate system.

Tools to the right of image are active when indicated by the blue bar. They allow you to:
- Pan
- Box zoom
- Scroll-wheel zoom
- Drop points, or select points and move them
- Save the resulting image  

#### Instructions 
Pan, zoom, and drop points at surveyed gcp locations. Edit by adjusting point locations by click and drag, if necessary.
To remove a point, edit the table to make that line blank.

Edit the third column to insert ids for gcps. (Double click, then single click to edit the cell).

After defining the points, move to the next cell and run it...the data should be appear in the `point_stream` object.

#### TODO
* Initialze without first line of no data
* Use alternative library to read image? Matplotlib?
* Optimize layout
* Add title to figure?

In [ ]:
# Load image with pixel values as bounds
# TODO: change to more common tool for loading images? matplotlib?
img_pil = Image.open(img_dir+img_fname)
img_height = img_pil.height
img_width = img_pil.width

bounds = (0, 0, img_width, img_height)
img = hv.RGB(np.array(img_pil), bounds=bounds)

# Create the PointDraw stream and link it to a Table
# TODO - Do this without first row full of NaNs
# (That was the only way I could make the last row the ID)
points = hv.Points(([np.nan], [np.nan], [None]), vdims='gcp_id')
point_stream = streams.PointDraw(data=points.columns(), source=points)
table = hv.Table(points, ['x', 'y'], 'gcp_id')

# This is how two columns are initialized
# points = hv.Points(([], []))
# point_stream = streams.PointDraw(data=points.columns(), source=points)
# table = hv.Table(points, ['x', 'y'])

#table.opts(width=300)
DataLink(points, table)
(img * points + table).opts(
    opts.RGB(data_aspect=1),
    opts.Layout(merge_tools=False),
    opts.Points(active_tools=['point_draw'], size=10, tools=['hover'],
                height=400, width=600),
    opts.Table(editable=True))

In [ ]:
# point_stream.data is a dict containing the tabulated values
gcp_id = np.array( point_stream.data['gcp_id'] )
Ud = np.array( point_stream.data['x'] )
Vd = img_height - np.array( point_stream.data['y'] ) # image convention starts in upper left corner
ok = np.squeeze(np.argwhere(np.isfinite(Ud+Vd)))
for i,(id, Udi, Vdi) in enumerate(zip(gcp_id[ok], Ud[ok], Vd[ok])):
    print('{}, {}, {:.02f}, {:.03f}'.format(i, id, Udi, Vdi))

#### Save the points in a .csv file

In [ ]:
with open( opath, 'w') as csv_file:
    for (id, Udi, Vdi) in zip(gcp_id[ok], Ud[ok], Vd[ok]):
       csv_file.write('{}, {:.02f}, {:.03f}\n'.format(id, Udi, Vdi))
csv_file.close()
print('Points saved in ',opath)